<a href="https://colab.research.google.com/github/Whereamiactually/lyceumcompling10/blob/main/NER_lemmatization_stemming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Распознавание именованных сущностей

Natasha также умеет распознавать **именованные сущности** (NER - Named Entity Recognition): имена людей, названия организаций, книг, городов и другие имена собственные.

За одной задачей NER, на самом деле, стоит две:
1. обнаружить, что какая-то последовательность слов — это именованная сущность;
2. понять, к какому классу (имя человека, название организации, город и т.п.) эта именованная сущность относится.

Какие классы именованных сущностей обычно хотят найти? Практически всегда стараются извлекать **имена людей и названия мест и организаций**, а дальше все зависит от конкретных задач, которые нужно решать, или от возможностей предобученной системы, которую планируется использовать. Также к задаче NER относят **извлечение дат, денежных сумм (число + валюта)**.

Всё это нужно, чтобы решать задачи референции, референциального выбора и кореференции и метонимии, которые являются центральными для поиска, вопросно-ответных систем, связности текста, синтаксического и морфологического парсинга и т.д.

Многие именованные сущности могут в разных контекстах относиться к разным классам: слово «Чехов» может быть человеком, городом, названием клуба и т.д.

Какие **проблемы** могут быть связаны с NER?

1. Именованные сущности редко состоят из одного слова. Например, из предложения «Звонил доктор Владимир Бомгард» нужно извлечь, как минимум, имя и фамилию — «Владимир Бомгард», а для многих задач полезно уметь находить полное «наименование» человека, о котором говорится: «доктор Владимир Бомгард».
2. Не всегда очевидны границы, например, словосочетание «Иван Васильевич и партнеры» может быть названием некоторой организации, а может подразумевать некоторого Ивана Васильевича и отдельно его партнеров.
3. С большой буквы пишутся не только имена собственные. Это верно и для русского, и еще больше для английского, но особенно хорошо видно в немецком, где с большой буквы пишутся вообще все существительные.
4. Не во всех языках вообще есть большие буквы (например, это верно для языков со слоговыми системами).

Как **работают** с именованными сущностями?

Хотя большинство именованных сущностей состоит из нескольких слов, при решении этой задачи обычно рассматривают отдельные слова и решают, является ли это слово частью именованной сущности или нет. При этом **различают начало, середину и конец именованной сущности**.

При разметке именованных сущностей принято использовать префиксы **BIOES-схемы**:
* **«B» (beginning, начало)** для обозначения первого слова,
* **«E» (end, конец)** для обозначения последнего слова,
* **«I» (intermediate, промежуточный)** для всех слов между первым и последним,
* **«S» (single, одиночный)** для обозначения именованной сущности, состоящей из одного слова,
* **«O» (outside, снаружи)** — слово не относится ни к какой сущности.

Задача сводится к **пословной классификации**. Обычно тренируют **нейросетевую модель** на большом количестве текстов с разметкой именованных сущностей. В основе моделей обычно стоят модули, которые позволяют учитывать контекст с двух сторон от анализируемого слова.

Хорошие результаты дают и **классические классификаторы**, работающие с предзаданным множеством признаков. Большие буквы или нестандартное использование больших и маленьких букв (iPhone), а также специфических символов (H&M) внутри слова — все это полезные признаки для выявления именованных сущностей.

Также до сих пор используются **системы, основанные на правилах**: в них прописываются разные шаблонные схемы именованных сущностей. Например, шаблон **«Министерство + образования\туризма\здравоохранения + Название страны»** поможет найти «Министерство образования Италии» и «Министерство туризма Мексики». Шаблон **«однозначное\двузначное число + название месяца + четырехзначное число»** — это дата. Основная проблема такого подхода: на подготовку правил требуется очень много времени, а малейшее отступление от паттерна (например, банальная опечатка) все сломает. Также набор правил пишется под определенный язык, поэтому набор правил, помогающий извлекать именованные сущности из текстов на русском языке никак не поможет при обработке другого языка.

**Активно используемых именованных сущностей конечное количество**, многие из них уже собраны в списки (так называемые **газетиры**, gazetteers): географические названия, словари имен, реестры организаций и т.п. Всю эту информацию тоже стараются использовать для решения задачи NER.

In [1]:
pip install natasha

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 91.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=efcadb602fc9d04c0478f882994e4db8412e2c63c3aa1069d04ae100975a5a9c
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26094 sha256=7da0a6bd9746b4ff1f6b59807df033819ba4835612c06e97bef1056ab08a7fb1
  Stored in directory: /root/.cache/pip/wheels/fa/80/8c/43488a924a046b733b64de3fac99252674c892a4c3801c0a61
Successfully built docopt intervaltree


In [30]:
from natasha import (
  Segmenter,

  NewsEmbedding,
  NewsMorphTagger,
  NewsSyntaxParser,

  Doc
)

segmenter = Segmenter() # сегментирует текст, нужно для распознавания именованных сущностей
emb = NewsEmbedding() # преобразует языковые сущности в числовой вектор
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)

In [74]:
text = 'Киллиан Мёрфи участвовал в 2007 году в кампании Rock the Voteruen, ориентированной на молодых избирателей на всеобщих выборах, и проводил кампанию за права бездомных с организацией Focus Ireland. В 2011 году он стал патроном Исследовательского центра ЮНЕСКО по проблемам детей и семьи при Ирландском национальном университете в Голуэе. Он тесно связан с работой профессора Пэта Долана, директора UCFRC и кафедры ЮНЕСКО по вопросам детей, молодежи и гражданского участия. В феврале 2012 года он написал послание поддержки бывшим работникам Vita Cortexruen, участвовавшим в сидячей забастовке на их заводе. Какое это имеет отношение к Высшей Школе Экономики? Александр Сергеевич Пушкин не понимает.'
doc = Doc(text) # текст нужно определённым образом преобразовать
doc.segment(segmenter) # сегментируем тест
doc.tag_morph(morph_tagger)
doc.parse_syntax(syntax_parser)

In [52]:
from natasha import NewsNERTagger
ner_tagger = NewsNERTagger(emb) # инициализируем распознаватель именованных сущностей

In [75]:
doc.tag_ner(ner_tagger)
doc.ner.print()

Киллиан Мёрфи участвовал в 2007 году в кампании Rock the Voteruen, 
PER──────────                                                      
ориентированной на молодых избирателей на всеобщих выборах, и проводил
 кампанию за права бездомных с организацией Focus Ireland. В 2011 году
                                            ORG──────────             
 он стал патроном Исследовательского центра ЮНЕСКО по проблемам детей 
                  ORG────────────────────── ORG───                    
и семьи при Ирландском национальном университете в Голуэе. Он тесно 
            ORG─────────────────────────────────   LOC───           
связан с работой профессора Пэта Долана, директора UCFRC и кафедры 
                            PER────────            ORG──           
ЮНЕСКО по вопросам детей, молодежи и гражданского участия. В феврале 
ORG───                                                               
2012 года он написал послание поддержки бывшим работникам Vita 
                               

In [76]:
from natasha import (
  MorphVocab,
  PER,
  NamesExtractor,
)

morph_vocab = MorphVocab()
names_extractor = NamesExtractor(morph_vocab)

Мы можем вывести нормализованные именованные сущности.

In [77]:
for span in doc.spans: # проходимся по всем именованным сущностям и добавляет к ним нормализацию
  span.normalize(morph_vocab)
{_.text: _.normal for _ in doc.spans if _.text != _.normal}

{'Киллиан Мёрфи': 'Киллиан Мерфи',
 'Исследовательского центра': 'Исследовательский центр',
 'Ирландском национальном университете': 'Ирландский национальный университет',
 'Голуэе': 'Голуэй',
 'Пэта Долана': 'Пэта Долан',
 'Высшей Школе Экономики': 'Высочайшая Школа Экономики'}

Мы можем отдельно выцеплять имена людей, причем мы можем их ФИ(О) поделить на имя и фамилию (и отчество). Но для этого нам нужно знать, в каком формате записано имя (например, ФИО или ИФ).

In [78]:
for span in doc.spans:
  if span.type == PER:
    span.extract_fact(names_extractor)

{_.normal: _.fact.as_dict for _ in doc.spans if _.type == PER}

{'Киллиан Мерфи': {'first': 'Киллиан', 'last': 'Мерфи'},
 'Пэта Долан': {'first': 'Пэта', 'last': 'Долан'},
 'Александр Сергеевич Пушкин': {'first': 'Александр',
  'last': 'Пушкин',
  'middle': 'Сергеевич'}}

In [79]:
from natasha import (
  DatesExtractor, # для дат
  MoneyExtractor, # для денежных сумм
  AddrExtractor # для адресов
)

dates_extractor = DatesExtractor(morph_vocab)
money_extractor = MoneyExtractor(morph_vocab)
address_extractor = AddrExtractor(morph_vocab)

In [88]:
list(dates_extractor(text))

[Match(
     start=27,
     stop=36,
     fact=Date(
         year=2007,
         month=None,
         day=None
     )
 ),
 Match(
     start=198,
     stop=207,
     fact=Date(
         year=2011,
         month=None,
         day=None
     )
 ),
 Match(
     start=473,
     stop=490,
     fact=Date(
         year=2012,
         month=2,
         day=None
     )
 )]

### Stanza

Это библиотека для обработки естественных языков (как и Natasha). Выполнять она может практически всё то же самое, что и Natasha (например, токенизация, лемматизация, морфологический анализ), только с большим количеством языков (66 языков, включая русский). Список языков, для которых возможно распознавание именованных сущностей, можно найти вот [тут](https://stanfordnlp.github.io/stanza/ner_models.html).

In [11]:
pip install stanza

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.2/881.2 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 33.6 MB/s eta 0:00:00


In [12]:
import stanza
stanza.download('en') # загружаем язык, с которым мы хотим работать

INFO:stanza:Downloading default packages for language: en (English) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.


Нам нужно задать процессоры, с которыми мы хотим работать. В данном случае, **чтобы распознать именованные сущности, нам надо вначале токенизировать текст**, а затем уже попытаться их найти. Список других процессов можно найти [тут](https://stanfordnlp.github.io/stanza/).

In [13]:
nlp = stanza.Pipeline(lang = 'en', processors = 'tokenize, ner')

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor | Package          |
--------------------------------
| tokenize  | combined         |
| ner       | ontonotes_charlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


Попробуем на тексте на английском языке. Список тегов можно найти [тут](https://stanfordnlp.github.io/stanza/ner_models.html) (для разных языков он разный).

In [68]:
eng_text = 'Cillian Murphy participated in the 2007 Rock the Vote Ireland campaign, targeting young voters for the general election, and campaigning for the rights of the homeless with the organisation Focus Ireland. In 2011, he became a patron of the UNESCO Child and Family Research Centre at the National University of Ireland Galway. He is closely associated with the work of Professor Pat Dolan Director UCFRC and UNESCO Chair in Children, Youth and Civic Engagement. In February 2012, he wrote a message of support to the former Vita Cortex workers involved in a sit-in at their plant, congratulating them for "highlighting [what] is hugely important to us all as a nation". Murphy was a supporter of the 2018 Irish referendum to repeal the eighth amendment of the constitution that restricted access to abortions.'
doc = nlp(eng_text)
print(*[f'entity: {ent.text}\ttype: {ent.type}' for sent in doc.sentences for ent in sent.ents], sep = '\n')

entity: Cillian Murphy	type: PERSON
entity: 2007	type: DATE
entity: Rock the Vote Ireland	type: ORG
entity: Focus Ireland	type: ORG
entity: 2011	type: DATE
entity: the UNESCO Child and Family Research Centre	type: ORG
entity: the National University of Ireland Galway	type: ORG
entity: Pat Dolan	type: PERSON
entity: UCFRC	type: ORG
entity: UNESCO	type: ORG
entity: Youth and Civic Engagement	type: ORG
entity: February 2012	type: DATE
entity: Vita Cortex	type: ORG
entity: Murphy	type: PERSON
entity: 2018	type: DATE
entity: Irish	type: NORP
entity: eighth	type: ORDINAL


Мы можем выводить BIOES NER теги для каждого токена, чтобы посмотреть, не упустила ли что-то наша программа.

In [69]:
print(*[f'token: {token.text}\tner: {token.ner}' for sent in doc.sentences for token in sent.tokens], sep = '\n')

token: Cillian	ner: B-PERSON
token: Murphy	ner: E-PERSON
token: participated	ner: O
token: in	ner: O
token: the	ner: O
token: 2007	ner: S-DATE
token: Rock	ner: B-ORG
token: the	ner: I-ORG
token: Vote	ner: I-ORG
token: Ireland	ner: E-ORG
token: campaign	ner: O
token: ,	ner: O
token: targeting	ner: O
token: young	ner: O
token: voters	ner: O
token: for	ner: O
token: the	ner: O
token: general	ner: O
token: election	ner: O
token: ,	ner: O
token: and	ner: O
token: campaigning	ner: O
token: for	ner: O
token: the	ner: O
token: rights	ner: O
token: of	ner: O
token: the	ner: O
token: homeless	ner: O
token: with	ner: O
token: the	ner: O
token: organisation	ner: O
token: Focus	ner: B-ORG
token: Ireland	ner: E-ORG
token: .	ner: O
token: In	ner: O
token: 2011	ner: S-DATE
token: ,	ner: O
token: he	ner: O
token: became	ner: O
token: a	ner: O
token: patron	ner: O
token: of	ner: O
token: the	ner: B-ORG
token: UNESCO	ner: I-ORG
token: Child	ner: I-ORG
token: and	ner: I-ORG
token: Family	ner: I-ORG
token:

Теперь проверим на русском!

In [70]:
stanza.download('ru')
nlp = stanza.Pipeline(lang = 'ru', processors = 'tokenize,ner')
doc = nlp(text)

INFO:stanza:Downloading default packages for language: ru (Russian) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| ner       | wikiner   |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [71]:
print(*[f'entity: {ent.text}\ttype: {ent.type}' for sent in doc.sentences for ent in sent.ents], sep = '\n')

entity: Киллиан Мёрфи	type: PER
entity: Rock the Voteruen	type: ORG
entity: Focus Ireland	type: ORG
entity: Исследовательского центра ЮНЕСКО	type: ORG
entity: Ирландском национальном университете в Голуэе	type: LOC
entity: Пэта Долана	type: PER
entity: UCFRC	type: ORG
entity: ЮНЕСКО	type: ORG
entity: Vita Cortexruen	type: ORG
entity: Высшей Школе Экономики	type: ORG
entity: Александр Сергеевич Пушкин	type: PER


### Ещё один распознаватель ИС

SpaCy - ещё одна библиотека для обработки естественных языков. [Здесь](https://spacy.io/usage) можно найти подробную информацию, в том числе посмотреть, какие языки поддерживает эта библиотека.

Посмотрим вначале на то, как она справляется с английским языком.

In [89]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.9/807.9 kB 44.3 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


  Obtaining dependency information for spacy from https://files.pythonhosted.org/packages/fa/7c/8518799f3fc85e4c2538c0b4fc2306de90696f00c1a8286c943f9414292a/spacy-3.7.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for weasel<0.4.0,>=0.1.0 from https://files.pythonhosted.org/packages/de/f5/6786a5fd1ab6a38511f3772c9002f312a2d509c1237ae514631adf145ad4/weasel-0.3.2-py3-none-any.whl.metadata
  Obtaining dependency information for cloudpathlib<0.16.0,>=0.7.0 from https://files.pythonhosted.org/packages/97/a2/e9a5bd762cccefc92a98c87354a65a8b75c280ab187a05e6d5851adbdae6/cloudpathlib-0.15.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.6.1
    Uninstalling sp

In [90]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [107]:
doc = nlp(eng_text)

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Cillian Murphy 0 14 PERSON
Rock the 40 48 EVENT
Vote Ireland 49 61 GPE
Focus Ireland 190 203 GPE
2011 208 212 DATE
the National University of Ireland Galway 283 324 ORG
Pat Dolan 378 387 PERSON
UCFRC 397 402 PERSON
UNESCO Chair 407 419 ORG
Children 423 431 GPE
February 2012 464 477 DATE
Vita Cortex 523 534 ORG
Murphy 669 675 PERSON
2018 699 703 DATE
Irish 704 709 NORP
eighth 735 741 ORDINAL


Мы можем вручную задать новые теги и соотнести определенные именованные сущности с ними.

In [108]:
from spacy.tokens import Span

ents = [(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents]
print('До:', ents)

rvi_ent = Span(doc, 16, 19, label = "ELECT") # создаем новый спан для слов 16-18
doc.ents = list(doc.ents) + [rvi_ent]

ents = [(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents]
print('После:', ents)

До: [('Cillian Murphy', 0, 14, 'PERSON'), ('Rock the', 40, 48, 'EVENT'), ('Vote Ireland', 49, 61, 'GPE'), ('Focus Ireland', 190, 203, 'GPE'), ('2011', 208, 212, 'DATE'), ('the National University of Ireland Galway', 283, 324, 'ORG'), ('Pat Dolan', 378, 387, 'PERSON'), ('UCFRC', 397, 402, 'PERSON'), ('UNESCO Chair', 407, 419, 'ORG'), ('Children', 423, 431, 'GPE'), ('February 2012', 464, 477, 'DATE'), ('Vita Cortex', 523, 534, 'ORG'), ('Murphy', 669, 675, 'PERSON'), ('2018', 699, 703, 'DATE'), ('Irish', 704, 709, 'NORP'), ('eighth', 735, 741, 'ORDINAL')]
После: [('Cillian Murphy', 0, 14, 'PERSON'), ('Rock the', 40, 48, 'EVENT'), ('Vote Ireland', 49, 61, 'GPE'), ('the general election', 99, 119, 'Elect'), ('Focus Ireland', 190, 203, 'GPE'), ('2011', 208, 212, 'DATE'), ('the National University of Ireland Galway', 283, 324, 'ORG'), ('Pat Dolan', 378, 387, 'PERSON'), ('UCFRC', 397, 402, 'PERSON'), ('UNESCO Chair', 407, 419, 'ORG'), ('Children', 423, 431, 'GPE'), ('February 2012', 464, 477, 

Можем красиво отобразить!

In [99]:
from spacy import displacy

In [109]:
displacy.render(doc,style = "ent",jupyter = True)

Либо что-то только одно!

In [110]:
opts = {'ents': ['PERSON']}
displacy.render(doc, style = "ent", jupyter = True, options = opts)

Ну и все то же самое для русского языка.

In [111]:
! python -m spacy download ru_core_news_sm

2023-10-08 13:10:55.518551: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 26.2 MB/s eta 0:00:00
  Obtaining dependency information for pymorphy3>=1.0.0 from https://files.pythonhosted.org/packages/d7/f9/ffb9afde503dc6bb2361ea79ceaea18138fbcee32aec4c5d8efa49180753/pymorphy3-1.2.1-py3-none-any.whl.metadata
  Obtaining dependency information for docopt-ng>=0.6 from https://files.pythonhosted.org/packages/6c/4a/c3b77fc1a24510b08918b43a473410c0168f6e657118807015f1f1edceea/docopt_ng-0.9.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 6.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [112]:
nlp = spacy.load("ru_core_news_sm")

In [113]:
doc = nlp(text)

for ent in doc.ents:
  print(ent.text, ent.start_char, ent.end_char, ent.label_)

Киллиан Мёрфи 0 13 PER
Rock the Voteruen 48 65 ORG
Focus Ireland 181 194 ORG
Исследовательского центра 225 250 ORG
ЮНЕСКО 251 257 ORG
Ирландском национальном университете 289 325 ORG
Голуэе 328 334 LOC
Пэта Долана 373 384 PER
UCFRC 396 401 ORG
ЮНЕСКО 412 418 ORG
Vita Cortexruen 539 554 ORG
Высшей Школе Экономики 633 655 ORG
Александр Сергеевич Пушкин 657 683 PER


In [114]:
displacy.render(doc, style = "ent", jupyter = True)

### Лемматизация и стемминг

Стемминг vs лемматизация: что это такое и в чём разница?

Это всё способы нормализации.

**Лемма** - словарная форма слова. **Лемматизация** учитывает морфологический анализ слов и использует словарь для **нахождения словарной формы** слова:

* для существительных — именительный падеж, единственное число;
* для прилагательных — именительный падеж, единственное число, мужской род;
* для глаголов, причастий, деепричастий — глагол в инфинитиве несовершенного вида.

**Стемминг** работает, не учитывая контекст и морфологические характеристики слова, Стемминг позволяет привести слово к его основной форме. Суть подхода **в нахождении основы слова, для этого с конца и начала слова последовательно отрезаются его части**. Правила отсекания для стеммера создаются заранее, и чаще всего представляют из себя регулярные выражения, что делает данный подход трудоемким. Вторым недостатком подхода является возможная потеря информации при отрезании частей, например, мы можем потерять информацию о части речи.

Natasha умеет решать задачу лемматизации. Для этого она использует другой морфологический анализатор **Pymorphy2**, с которым мы в полной мере познакомимся чуть попозже.

Чтобы посмотреть, как работает **стемминг**, мы посмотрим на SnowballStemmer из встроенной питоновской библиотеки **NLTK**.

**Natural Language Toolkit** (набор инструментов для естественной обработки языка) - пакет библиотек и программ для обработки естественного языка (токенизация, стемминг, лемматизация, POS-тэггинг, графическое представление структуры, семантические рассуждения, ...). Подробнее мы с ней тоже познакомимся позже.

In [137]:
import nltk
from nltk.stem import SnowballStemmer
snowball = SnowballStemmer(language = "russian")

In [138]:
stems = {}
for token in doc.tokens:
  stem = snowball.stem(token.text)
  stems[token.text] = stem

stems

{'Киллиан': 'киллиа',
 'Мёрфи': 'мерф',
 'участвовал': 'участвова',
 'в': 'в',
 '2007': '2007',
 'году': 'год',
 'кампании': 'кампан',
 'Rock': 'Rock',
 'the': 'the',
 'Voteruen': 'Voteruen',
 ',': ',',
 'ориентированной': 'ориентирова',
 'на': 'на',
 'молодых': 'молод',
 'избирателей': 'избирател',
 'всеобщих': 'всеобщ',
 'выборах': 'выбор',
 'и': 'и',
 'проводил': 'провод',
 'кампанию': 'кампан',
 'за': 'за',
 'права': 'прав',
 'бездомных': 'бездомн',
 'с': 'с',
 'организацией': 'организац',
 'Focus': 'Focus',
 'Ireland': 'Ireland',
 '.': '.',
 'В': 'в',
 '2011': '2011',
 'он': 'он',
 'стал': 'стал',
 'патроном': 'патрон',
 'Исследовательского': 'исследовательск',
 'центра': 'центр',
 'ЮНЕСКО': 'юнеск',
 'по': 'по',
 'проблемам': 'проблем',
 'детей': 'дет',
 'семьи': 'сем',
 'при': 'при',
 'Ирландском': 'ирландск',
 'национальном': 'национальн',
 'университете': 'университет',
 'Голуэе': 'голуэ',
 'Он': 'он',
 'тесно': 'тесн',
 'связан': 'связа',
 'работой': 'работ',
 'профессора': '

In [126]:
from natasha import MorphVocab

morph_vocab = MorphVocab()

text = 'Киллиан Мёрфи участвовал в 2007 году в кампании Rock the Voteruen, ориентированной на молодых избирателей на всеобщих выборах, и проводил кампанию за права бездомных с организацией Focus Ireland. В 2011 году он стал патроном Исследовательского центра ЮНЕСКО по проблемам детей и семьи при Ирландском национальном университете в Голуэе. Он тесно связан с работой профессора Пэта Долана, директора UCFRC и кафедры ЮНЕСКО по вопросам детей, молодежи и гражданского участия. В феврале 2012 года он написал послание поддержки бывшим работникам Vita Cortexruen, участвовавшим в сидячей забастовке на их заводе. Какое это имеет отношение к Высшей Школе Экономики? Александр Сергеевич Пушкин не понимает.'
doc = Doc(text)
doc.segment(segmenter)
doc.tag_morph(morph_tagger)

for token in doc.tokens:
  token.lemmatize(morph_vocab)

{word.text: word.lemma for word in doc.tokens} # создаёт для каждого токена вхождение в словаре и соотносит его с словарной формой

{'Киллиан': 'киллиан',
 'Мёрфи': 'мерфь',
 'участвовал': 'участвовать',
 'в': 'в',
 '2007': '2007',
 'году': 'год',
 'кампании': 'кампания',
 'Rock': 'rock',
 'the': 'the',
 'Voteruen': 'voteruen',
 ',': ',',
 'ориентированной': 'ориентировать',
 'на': 'на',
 'молодых': 'молодой',
 'избирателей': 'избиратель',
 'всеобщих': 'всеобщий',
 'выборах': 'выбор',
 'и': 'и',
 'проводил': 'проводить',
 'кампанию': 'кампания',
 'за': 'за',
 'права': 'право',
 'бездомных': 'бездомный',
 'с': 'с',
 'организацией': 'организация',
 'Focus': 'focus',
 'Ireland': 'ireland',
 '.': '.',
 'В': 'в',
 '2011': '2011',
 'он': 'он',
 'стал': 'стать',
 'патроном': 'патрон',
 'Исследовательского': 'исследовательский',
 'центра': 'центр',
 'ЮНЕСКО': 'юнеско',
 'по': 'по',
 'проблемам': 'проблема',
 'детей': 'ребенок',
 'семьи': 'семья',
 'при': 'при',
 'Ирландском': 'ирландский',
 'национальном': 'национальный',
 'университете': 'университет',
 'Голуэе': 'голуэй',
 'Он': 'он',
 'тесно': 'тесно',
 'связан': 'связа

Лемматизиация не очень хорошо справляется с названиями именованных сущностей, потому что она их делит пословно. Но это можно исправить (мы уже делали то же самое выше).

In [127]:
from natasha import NewsNERTagger

ner_tagger = NewsNERTagger(emb) # инициализируем распознаватель именованных сущностей
doc.tag_ner(ner_tagger)

for span in doc.spans: # эта функция опирается на морфологический и синатаксический анализ, а также на NER
  span.normalize(morph_vocab)
{span.text: span.normal for span in doc.spans}

{'Киллиан Мёрфи': 'Киллиан Мерфи',
 'Focus Ireland': 'Focus Ireland',
 'Исследовательского центра': 'Исследовательский центр',
 'ЮНЕСКО': 'ЮНЕСКО',
 'Ирландском национальном университете': 'Ирландский национальный университет',
 'Голуэе': 'Голуэй',
 'Пэта Долана': 'Пэта Долан',
 'UCFRC': 'UCFRC',
 'Vita Cortexruen': 'Vita Cortexruen',
 'Высшей Школе Экономики': 'Высочайшая Школа Экономика',
 'Александр Сергеевич Пушкин': 'Александр Сергеевич Пушкин'}